In [1]:
import numpy as np
import cv2
import math
import os
import glob
import re
from sklearn import svm

## Part1

In [2]:
#Computes the center (a,b) of a circle passing through point (x,y) with a given radius and angle of gradient.
def calculate_vote(x, y, radius, direc):
    a = int(x - radius * math.cos(direc))
    b = int(y - radius * math.sin(direc))
    return a, b

In [3]:
#Checks if the calculated circle center (a,b) is within the bounds of the hough_space dimensions.
def validate_vote(a, b, hough_space_shape):
    return a >= 0 and a < hough_space_shape[1] and b >= 0 and b < hough_space_shape[0]

In [4]:
#Performs the Hough Circle Transform to detect circles in an edge-detected image, returning a 3D Hough space array.
def hough_transform(edge_image, mag, direc, radius_range):
    hough_space = np.zeros((edge_image.shape[0], edge_image.shape[1], len(radius_range)))
    for y, x in np.ndindex(edge_image.shape):
        if edge_image[y, x] > 0:
            for radius_index, radius in enumerate(radius_range):
                a, b = calculate_vote(x, y, radius, direc[y, x])
                if validate_vote(a, b, hough_space.shape):
                    hough_space[b, a, radius_index] += 1

    return hough_space

In [5]:
#Applies non-maximum suppression to filter out circles that are too close to each other, based on a distance threshold.
def nonmax_supp(detected_circles, distance_threshold):
    sorted_circles = sorted(detected_circles, key=lambda x: x[2], reverse=True)  # Sort by accumulator value
    nonmax_circles = []
    for target in sorted_circles:
        x, y, radius, _ = target
        if all(math.hypot(x - cx, y - cy) >= distance_threshold * (radius + cr) / 2 for cx, cy, cr, _ in nonmax_circles):
            nonmax_circles.append(target)

    return nonmax_circles

In [6]:
# Processes an image by resizing it, detecting edges, finding circles using Hough Transform, 
#applying non-maximum suppression, drawing the circles, padding the image to a square, and saving the results.

def process_image(image_path, output_dir, edge_output_dir, distance_threshold):
    original_image = cv2.imread(image_path, cv2.IMREAD_COLOR)
    max_size, canny_thresholds, radius_range = (200, (150, 200), np.arange(30, 100)) if 'Train' in image_path else (1000, (50, 150), np.arange(5, 170))

    height, width = original_image.shape[:2]
    scale = min(max_size / height, max_size / width)
    if scale < 1:
        new_dimensions = (int(width * scale), int(height * scale))
        resized_image = cv2.resize(original_image, new_dimensions, interpolation=cv2.INTER_AREA)
    else:
        resized_image = original_image
    output_image = resized_image.copy()
    gray_image = cv2.cvtColor(resized_image, cv2.COLOR_BGR2GRAY)
    blurred_image = cv2.GaussianBlur(gray_image, (3,3), 0)
    edges = cv2.Canny(blurred_image, *canny_thresholds)

    edge_image_name = os.path.splitext(os.path.basename(image_path))[0] + '_Edge.jpg'
    edge_image_path = os.path.join(edge_output_dir, edge_image_name)
    cv2.imwrite(edge_image_path, edges)

    sobel_x = cv2.Sobel(blurred_image, cv2.CV_64F, 1, 0, ksize=3)
    sobel_y = cv2.Sobel(blurred_image, cv2.CV_64F, 0, 1, ksize=3)
    mag, direc = np.sqrt(sobel_x**2 + sobel_y**2), np.arctan2(sobel_y, sobel_x)

    hough_space = hough_transform(edges, mag, direc, radius_range)
    threshold = 0.5 * hough_space.max()
    detected_circles = [(x, y, radius_range[r], hough_space[y, x, r]) for y, x, r in zip(*np.where(hough_space >= threshold))]
    nonmax_circles = nonmax_supp(detected_circles, distance_threshold)

    scale = min(max_size / height, max_size / width)

    if scale < 1:
        new_dimensions = (int(width * scale), int(height * scale))
        output_image = cv2.resize(original_image, new_dimensions, interpolation=cv2.INTER_AREA)
    else:
        output_image = original_image.copy()

    for x, y, radius, _ in nonmax_circles:
        cv2.circle(output_image, (x, y), radius, (0, 0, 256), 3)
        for x, y, radius, _ in nonmax_circles:
            cv2.circle(output_image, (x, y), radius, (0, 0, 256), 3)
    # Calculate and apply padding
    height, width = output_image.shape[:2]
    del_h, del_w = max_size - height, max_size - width
    pad_top, pad_bottom = del_h // 2, del_h - del_h // 2
    pad_left, pad_right = del_w // 2, del_w - del_w // 2
    pad_image = cv2.copyMakeBorder(output_image, pad_top, pad_bottom, pad_left, pad_right, cv2.BORDER_CONSTANT, value=(256, 256, 256))

    cv2.imwrite(os.path.join(output_dir, os.path.basename(image_path)), pad_image)

In [7]:
#Iterates through given directory sets, cleans output folders, and processes images for edge and circle detection.
def process_directories(input_dirs, output_dirs, edge_output_dirs):
    for input_dir, output_dir, edge_output_dir in zip(input_dirs, output_dirs, edge_output_dirs):
        # Set distance threshold based on input directory
        distance_threshold = 7 if input_dir == 'Train' else 2
        # Clean output directories
        for directory in [output_dir, edge_output_dir]:
            if not os.path.exists(directory):
                os.makedirs(directory)
            for file in glob.glob(os.path.join(directory, '*.jpg')):
                os.remove(file)
        # Process all image paths in the input directory
        image_paths = glob.glob(os.path.join(input_dir, '*.jpg'))
        for image_path in image_paths:
            process_image(image_path, output_dir, edge_output_dir, distance_threshold)

# Define input and output directories
input_dirs = ['Train', 'TestV', 'TestR']
output_dirs = ['Train_Hough', 'TestV_Hough', 'TestR_Hough']
edge_output_dirs = ['Train_Edge', 'TestV_Edge', 'TestR_Edge']

# Process all directories
process_directories(input_dirs, output_dirs, edge_output_dirs)

### Part2

In [8]:
#Reads an image, converts it to grayscale, resizes to a maximum size, pads it to be square, and returns the 
#processed image along with a processed part of the filename.
def process_image2(image_path, filename):
    # Read the image and convert to grayscale
    original_image = cv2.imread(image_path, cv2.IMREAD_COLOR)
    gray_image = cv2.cvtColor(original_image, cv2.COLOR_BGR2GRAY)

    max_size = 200
    height, width = gray_image.shape[:2]
    scale = min(max_size / height, max_size / width)

    if scale < 1:
        new_dimensions = (int(width * scale), int(height * scale))
        resized_image = cv2.resize(gray_image, new_dimensions, interpolation=cv2.INTER_AREA)
    else:
        resized_image = gray_image

    height, width = resized_image.shape[:2]
    _height = max_size - height
    _width = max_size - width
    pad_top = _height // 2
    pad_bottom = _height - pad_top
    pad_left = _width // 2
    pad_right = _width - pad_left

    padded_image = cv2.copyMakeBorder(resized_image, pad_top, pad_bottom, pad_left, pad_right, cv2.BORDER_CONSTANT, value=(256, 256, 256))

    # Process filename
    parts = filename.split('_')
    if len(parts) > 1:
        return padded_image, parts[0] + '_' + parts[1].split('.')[0]

    return padded_image, None

In [9]:
#Creates histograms of gradient orientations within cells, normalizes them in blocks, and concatenates to form 
#the HOG feature vector.
def create_and_normalize_histograms(mag, direc, cell_size=32, bin_size=9, block_size=3):
    cell_rows, cell_cols = mag.shape[0] // cell_size, mag.shape[1] // cell_size
    histograms = np.zeros((cell_rows, cell_cols, 180 // bin_size))
    val = 1e-7  # Small value to avoid division by zero
    normalized_histograms = []
    for i in range(cell_rows):
        for j in range(cell_cols):
            cell_slice = (slice(i * cell_size, (i + 1) * cell_size), slice(j * cell_size, (j + 1) * cell_size))
            cell_mag = mag[cell_slice]
            cell_direc = direc[cell_slice]
            histograms[i, j] = np.histogram(cell_direc, bins=np.arange(0, 181, bin_size), weights=cell_mag)[0]
    for i in range(histograms.shape[0] - block_size + 1):
        for j in range(histograms.shape[1] - block_size + 1):
            block = histograms[i:i+block_size, j:j+block_size].flatten()
            normalized = block / np.sqrt(np.sum(block**2) + val)
            normalized_histograms.append(normalized)

    return np.concatenate(normalized_histograms)

In [10]:
#calculates gradient magnitudes and directions, then extracts Histogram of Oriented Gradients (HOG)
#features using these gradients.
def calculate_hog(image, cell_size=32, bin_size=9, block_size=3): 
    grad_x = cv2.Sobel(image, cv2.CV_64F, 1, 0, ksize=3)
    grad_y = cv2.Sobel(image, cv2.CV_64F, 0, 1, ksize=3)

    mag = np.hypot(grad_x, grad_y)
    direc = np.degrees(np.arctan2(grad_y, grad_x)) % 180

    hog_features = create_and_normalize_histograms(mag, direc, cell_size=cell_size, bin_size=bin_size, block_size=block_size)
    return hog_features

In [11]:
#Enhances contrast, resizes, blurs, and applies edge detection to an image, then uses Hough Transform and non-maximum 
#suppression to detect and return circles.
def find_circle(image_path, threshold):
    original_image = cv2.imread(image_path, cv2.IMREAD_COLOR)

    max_size = 1000
    height, width = original_image.shape[:2]
    scale = min(max_size / height, max_size / width)
    if scale < 1:
        new_dimensions = (int(width * scale), int(height * scale))
        resized_image = cv2.resize(original_image, new_dimensions, interpolation=cv2.INTER_AREA)
    else:
        resized_image = original_image
    gray_image = cv2.cvtColor(resized_image, cv2.COLOR_BGR2GRAY)
    blurred_image = cv2.GaussianBlur(gray_image, (3, 3), 0)
    edges = cv2.Canny(blurred_image, 50, 150)

    sobel_x = cv2.Sobel(blurred_image, cv2.CV_64F, 1, 0, ksize=3)
    sobel_y = cv2.Sobel(blurred_image, cv2.CV_64F, 0, 1, ksize=3)
    mag, direc = np.sqrt(sobel_x**2 + sobel_y**2), np.arctan2(sobel_y, sobel_x)

    hough_space = hough_transform(edges, mag, direc, np.arange(5, 170))
    threshold_value = 0.5 * hough_space.max()
    circles = np.where(hough_space >= threshold_value)

    detected_circles = [(x, y, np.arange(5, 170)[r], hough_space[y, x, r]) for y, x, r in zip(*circles)]
    nonmax_circles = nonmax_supp(detected_circles, threshold)

    return nonmax_circles

In [12]:
# Initialize empty lists for HOG features and labels
features = []
labels = []

train_dir = 'Train'
image_paths = glob.glob(os.path.join(train_dir, '*.jpg'))
for image_path in image_paths:
    image,label = process_image2(image_path,os.path.basename(image_path))
    feature = calculate_hog(image)
    features.append(feature)
    labels.append(label)

X = np.array(features)
y = np.array(labels)
classifier = svm.SVC()
classifier.fit(X, y)

SVC()

In [13]:
# This code block processes images from 'TestV' and 'TestR' directories by resizing, detecting circles, extracting 
#HOG features for each circle, classifying them with a pre-trained SVM classifier, and writing the predicted labels on the images.
test_dirs = ['TestV', 'TestR']
output_dirs = ['TestV_HoG', 'TestR_HoG']

for input_dir, output_dir in zip(test_dirs, output_dirs):
    os.makedirs(output_dir, exist_ok=True)
    for file in glob.glob(os.path.join(output_dir, '*.jpg')):
        os.remove(file)

    image_paths = glob.glob(os.path.join(input_dir, '*.jpg'))
    for image_path in image_paths:
        original_image = cv2.imread(image_path, cv2.IMREAD_COLOR)
        max_size = 1000
        height, width = original_image.shape[:2]
        scale = min(max_size / height, max_size / width)
        if scale < 1:
            new_dimensions = (int(width * scale), int(height * scale))
            resized_image = cv2.resize(original_image, new_dimensions, interpolation=cv2.INTER_AREA)
        else:
            resized_image = original_image.copy()

        circles = find_circle(image_path, threshold=0.5)

        for (x, y, radius, _) in circles:
            cv2.circle(resized_image, (x, y), radius, (0, 0, 256), 3)
            x1 = max(0, x - radius)
            y1 = max(0, y - radius)
            x2 = min(x + radius, resized_image.shape[1])
            y2 = min(y + radius, resized_image.shape[0])            
            region_of_interest = resized_image[y1:y2, x1:x2]

            region_of_interest_height, region_of_interest_width = region_of_interest.shape[:2]
            delta = abs(region_of_interest_height - region_of_interest_width)
            padding = delta // 2
            top, bottom, left, right = (padding, padding + delta % 2, padding, padding) if region_of_interest_height > region_of_interest_width else (padding, padding, padding + delta % 2, padding)
            squared_region_of_interest = cv2.copyMakeBorder(region_of_interest, top, bottom, left, right, cv2.BORDER_CONSTANT, value=(256, 256, 256))
            processed_region_of_interest = cv2.resize(squared_region_of_interest, (200, 200), interpolation=cv2.INTER_AREA)

            hog_features = calculate_hog(processed_region_of_interest).reshape(1, -1)
            class_name = classifier.predict(hog_features)[0]

            text_size = cv2.getTextSize(class_name, cv2.FONT_HERSHEY_SIMPLEX, 0.8, 1)[0]
            text_position = ((x, y - 25 - radius)[0] - text_size[0] // 2, (x, y - 25 - radius)[1] + text_size[1] // 2)
            cv2.putText(resized_image, class_name, text_position, cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 0), 1, cv2.LINE_AA)
            cv2.putText(resized_image, class_name, text_position, cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 256), 1, cv2.LINE_AA)
        cv2.imwrite(os.path.join(output_dir, os.path.basename(image_path)), resized_image)

## Just for try, knn k=3 

In [14]:
from sklearn.neighbors import KNeighborsClassifier
# Initialize empty lists for HOG features and labels
features = []
labels = []

train_dir = 'Train'
image_paths = glob.glob(os.path.join(train_dir, '*.jpg'))
for image_path in image_paths:
    image, label = process_image2(image_path, os.path.basename(image_path))
    feature = calculate_hog(image)
    features.append(feature)
    labels.append(label)

X = np.array(features)
y = np.array(labels)

# Initialize and train the KNN classifier
knn_classifier = KNeighborsClassifier(n_neighbors=3)  # n_neighbors can be tuned
knn_classifier.fit(X, y)


KNeighborsClassifier(n_neighbors=3)

In [15]:

# Processing test directories with the trained KNN classifier
test_dirs = ['TestV', 'TestR']
output_dirs = ['TestV_Knn', 'TestR_Knn']

for input_dir, output_dir in zip(test_dirs, output_dirs):
    os.makedirs(output_dir, exist_ok=True)
    for file in glob.glob(os.path.join(output_dir, '*.jpg')):
        os.remove(file)

    image_paths = glob.glob(os.path.join(input_dir, '*.jpg'))
    for image_path in image_paths:
        original_image = cv2.imread(image_path, cv2.IMREAD_COLOR)
        max_size = 1000
        height, width = original_image.shape[:2]
        scale = min(max_size / height, max_size / width)
        if scale < 1:
            new_dimensions = (int(width * scale), int(height * scale))
            resized_image = cv2.resize(original_image, new_dimensions, interpolation=cv2.INTER_AREA)
        else:
            resized_image = original_image.copy()

        circles = find_circle(image_path, threshold=0.5)

        for (x, y, radius, _) in circles:
            cv2.circle(resized_image, (x, y), radius, (0, 0, 256), 3)
            x1 = max(0, x - radius)
            y1 = max(0, y - radius)
            x2 = min(x + radius, resized_image.shape[1])
            y2 = min(y + radius, resized_image.shape[0])            
            region_of_interest = resized_image[y1:y2, x1:x2]

            region_of_interest_height, region_of_interest_width = region_of_interest.shape[:2]
            delta = abs(region_of_interest_height - region_of_interest_width)
            padding = delta // 2
            top, bottom, left, right = (padding, padding + delta % 2, padding, padding) if region_of_interest_height > region_of_interest_width else (padding, padding, padding + delta % 2, padding)
            squared_region_of_interest = cv2.copyMakeBorder(region_of_interest, top, bottom, left, right, cv2.BORDER_CONSTANT, value=(256, 256, 256))
            processed_region_of_interest = cv2.resize(squared_region_of_interest, (200, 200), interpolation=cv2.INTER_AREA)


          
            hog_features = calculate_hog(processed_region_of_interest).reshape(1, -1)
            class_name = knn_classifier.predict(hog_features)[0]

        
            text_size = cv2.getTextSize(class_name, cv2.FONT_HERSHEY_SIMPLEX, 0.8, 1)[0]
            text_position = ((x, y - 25 - radius)[0] - text_size[0] // 2, (x, y - 25 - radius)[1] + text_size[1] // 2)
            cv2.putText(resized_image, class_name, text_position, cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 0), 1, cv2.LINE_AA)
            cv2.putText(resized_image, class_name, text_position, cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 256), 1, cv2.LINE_AA)
        cv2.imwrite(os.path.join(output_dir, os.path.basename(image_path)), resized_image)